# Klassifikation mit LSTM

In [1]:
import os
import sys
import time
import numpy as np
from itertools import product

from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.wrappers.scikit_learn import KerasClassifier

from sklearn.model_selection import GridSearchCV, ShuffleSplit
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import tensorflow as tf

Using TensorFlow backend.


In [2]:
random_seed = 42

np.random.seed(random_seed)
tf.set_random_seed(random_seed)

In [3]:
from gesture_comparison_helper import load_gestures
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from Utils.data_preparation_helper import load_data_for_classification, prepare_data

c:\python36\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


FRAME_NUMBER = 3

acc_train_input, acc_train_label = prepare_data(train, FRAME_NUMBER, 'accelerometer')
acc_valid_input, acc_valid_label = prepare_data(valid, FRAME_NUMBER, 'accelerometer')

encoder = LabelEncoder()

X_train = acc_train_input.reshape(acc_train_input.shape[0], 1, acc_train_input.shape[1])
#y_train = acc_train_label.reshape(acc_train_label.shape[0], 1)
y_train = encoder.fit_transform(acc_train_label)
#print(y_train)

X_valid = acc_valid_input.reshape(acc_valid_input.shape[0], 1, acc_valid_input.shape[1])
y_valid = encoder.fit_transform(acc_valid_label)
#print(y_valid)

#print(encoder.classes_)

model = Sequential()
model.add(LSTM(X_train.shape[2], input_shape=(1, X_train.shape[2])))
model.add(Dense(len(encoder.classes_), activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
history = model.fit(X_train, y_train, epochs=10, batch_size=3) # X_train.shape[2])


predictions_proba = model.predict(X_valid)
predictions = model.predict_classes(X_valid, verbose=0)
#print()
#print(predictions_proba)

loss, accuracy = model.evaluate(X_valid, y_valid)
print()
print(loss)
print(accuracy)
print(predictions)

print(confusion_matrix(y_valid, predictions))

print(accuracy_score(y_valid, predictions))

print(classification_report(y_valid, predictions))

In [4]:
def create_model(n_classes=1, shape=1, num_units=1, optimizer='adam'):
    # create model
    model = Sequential()
    model.add(LSTM(units=num_units, input_shape=(1, shape)))
    model.add(Dense(n_classes, activation='sigmoid'))
    # Compile model
    model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

In [5]:
start_time = time.time()

optimizer = ['Adam']
batch_size = [1, 10, 100]
epochs = [10, 100, 300]
#lstm_units = [1, 10, 30, 50, 70, 100]

#parameters = [{'batch_size':batch_size, 'epochs':epochs, 'num_units':lstm_units, 'optimizer':optimizer}]
parameters = [{'batch_size':batch_size, 'epochs':epochs, 'optimizer':optimizer}]

frame_numbers = list(range(1,13))

cross_valid = ShuffleSplit(n_splits=1, test_size=0.2, random_state=random_seed)
verbose_level = 3

## Parametertest - own

In [7]:
train, valid, test = load_data_for_classification('own')

In [6]:
# lists will be filled with (accuracy, frame_number, params)
own_acc = (0, 0)
own_gyro = (0, 0)
own_ori = (0, 0)
own_emg = (0, 0)

### Accelerometer

In [7]:
def do_it(X_train, y_train, X_valid, y_valid, n_classes, frames, epochs, batch_size):
    lstm_model = create_model(n_classes=n_classes, shape=X_train.shape[2], num_units=X_train.shape[2])
    lstm_model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=0)

    prediction = lstm_model.predict_classes(X_valid, verbose=0)
    
    result = (accuracy_score(y_valid, prediction), frames, epochs, batch_size)
    return result

In [12]:
start_time = time.time()

for nr in frame_numbers:
    print(nr)
    try:
        acc_train_input, acc_train_label = prepare_data(train, nr, 'accelerometer', verbose=False)
        acc_valid_input, acc_valid_label = prepare_data(valid, nr, 'accelerometer', verbose=False)
    except ValueError as err:
        pass
    
    encoder = LabelEncoder()
    
    X_train = acc_train_input.reshape(acc_train_input.shape[0], 1, acc_train_input.shape[1])
    y_train = encoder.fit_transform(acc_train_label)
    X_valid = acc_valid_input.reshape(acc_valid_input.shape[0], 1, acc_valid_input.shape[1])
    y_valid = encoder.fit_transform(acc_valid_label)
    
    #for parameters in product(epochs, batch_size):
        #print('{} -- {}'.format(nr, parameters))
        #print(parameters)

    parameters[0]['n_classes'] = [len(encoder.classes_)]
    parameters[0]['shape'] = [X_train.shape[2]]
    parameters[0]['num_units'] = [X_train.shape[2]]

        #parameters[0]['batch_size'] = [1, 10, 100]
        #parameters[0]['batch_size'].append(X_train.shape[2])

    lstm_model = KerasClassifier(create_model, verbose=0)

    lstm = GridSearchCV(lstm_model, parameters, scoring='accuracy', cv=cross_valid,
                           verbose=verbose_level).fit(np.append(X_train, X_valid, axis=0),np.append(y_train, y_valid))
        
        #lstm_model = create_model(n_classes=len(encoder.classes_), shape=X_train.shape[2], num_units=X_train.shape[2])
        #lstm_model.fit(X_train, y_train, epochs=parameters[0], batch_size=parameters[1], verbose=0)

        #prediction = lstm_model.predict_classes(X_valid, verbose=0)

        #result = (accuracy_score(y_valid, prediction), nr, epochs, batch_size)
    result = (lstm.best_score_, nr, lstm.best_params_)
        
        #own_acc.append(do_it(X_train, y_train, X_valid, y_valid, len(encoder.classes_), nr, parameters[0], parameters[1]))
    if result[0] > own_acc[0]:
        own_acc = result

    

print('Duration: {}'.format(time.time() - start_time))

1
Fitting 1 folds for each of 9 candidates, totalling 9 fits
[CV] batch_size=1, epochs=10, n_classes=6, num_units=15, optimizer=Adam, shape=15 
[CV]  batch_size=1, epochs=10, n_classes=6, num_units=15, optimizer=Adam, shape=15, score=0.4423076923076923, total=  20.4s
[CV] batch_size=1, epochs=100, n_classes=6, num_units=15, optimizer=Adam, shape=15 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   20.7s remaining:    0.0s


[CV]  batch_size=1, epochs=100, n_classes=6, num_units=15, optimizer=Adam, shape=15, score=0.75, total= 2.7min
[CV] batch_size=1, epochs=300, n_classes=6, num_units=15, optimizer=Adam, shape=15 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  3.0min remaining:    0.0s


[CV]  batch_size=1, epochs=300, n_classes=6, num_units=15, optimizer=Adam, shape=15, score=0.8076923076923077, total= 8.1min
[CV] batch_size=10, epochs=10, n_classes=6, num_units=15, optimizer=Adam, shape=15 
[CV]  batch_size=10, epochs=10, n_classes=6, num_units=15, optimizer=Adam, shape=15, score=0.17307692307692307, total=   4.0s
[CV] batch_size=10, epochs=100, n_classes=6, num_units=15, optimizer=Adam, shape=15 
[CV]  batch_size=10, epochs=100, n_classes=6, num_units=15, optimizer=Adam, shape=15, score=0.5769230769230769, total=  16.9s
[CV] batch_size=10, epochs=300, n_classes=6, num_units=15, optimizer=Adam, shape=15 
[CV]  batch_size=10, epochs=300, n_classes=6, num_units=15, optimizer=Adam, shape=15, score=0.7115384615384616, total=  46.9s
[CV] batch_size=100, epochs=10, n_classes=6, num_units=15, optimizer=Adam, shape=15 
[CV]  batch_size=100, epochs=10, n_classes=6, num_units=15, optimizer=Adam, shape=15, score=0.19230769230769232, total=   3.0s
[CV] batch_size=100, epochs=100

...

[CV]  batch_size=1, epochs=100, n_classes=6, num_units=180, optimizer=Adam, shape=180, score=0.8846153846153846, total= 3.0min
[CV] batch_size=1, epochs=300, n_classes=6, num_units=180, optimizer=Adam, shape=180 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  3.4min remaining:    0.0s


[CV]  batch_size=1, epochs=300, n_classes=6, num_units=180, optimizer=Adam, shape=180, score=0.8653846153846154, total= 8.5min
[CV] batch_size=10, epochs=10, n_classes=6, num_units=180, optimizer=Adam, shape=180 
[CV]  batch_size=10, epochs=10, n_classes=6, num_units=180, optimizer=Adam, shape=180, score=0.7884615384615384, total=  12.6s
[CV] batch_size=10, epochs=100, n_classes=6, num_units=180, optimizer=Adam, shape=180 
[CV]  batch_size=10, epochs=100, n_classes=6, num_units=180, optimizer=Adam, shape=180, score=0.8846153846153846, total=  27.6s
[CV] batch_size=10, epochs=300, n_classes=6, num_units=180, optimizer=Adam, shape=180 
[CV]  batch_size=10, epochs=300, n_classes=6, num_units=180, optimizer=Adam, shape=180, score=0.8846153846153846, total= 1.0min
[CV] batch_size=100, epochs=10, n_classes=6, num_units=180, optimizer=Adam, shape=180 
[CV]  batch_size=100, epochs=10, n_classes=6, num_units=180, optimizer=Adam, shape=180, score=0.40384615384615385, total=  11.0s
[CV] batch_siz

[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed: 14.3min finished


Duration: 14082.516303777695


In [21]:
print(own_acc, end='\n')

(0.98076923076923073, 4, {'batch_size': 1, 'epochs': 100, 'n_classes': 6, 'num_units': 60, 'optimizer': 'Adam', 'shape': 60})


### Gyroskop

In [13]:
for nr in frame_numbers:
    print(nr)
    try:
        gyro_train_input, gyro_train_label = prepare_data(train, nr, 'gyro', verbose=False)
        gyro_valid_input, gyro_valid_label = prepare_data(valid, nr, 'gyro', verbose=False)
    except ValueError as err:
        pass
    
    encoder = LabelEncoder()

    X_train = gyro_train_input.reshape(gyro_train_input.shape[0], 1, gyro_train_input.shape[1])
    y_train = encoder.fit_transform(gyro_train_label)
    X_valid = gyro_valid_input.reshape(gyro_valid_input.shape[0], 1, gyro_valid_input.shape[1])
    y_valid = encoder.fit_transform(gyro_valid_label)

    parameters[0]['n_classes'] = [len(encoder.classes_)]
    parameters[0]['shape'] = [X_train.shape[2]]
    parameters[0]['num_units'] = [X_train.shape[2]]

    #parameters[0]['batch_size'] = [1, 10, 100]
    #parameters[0]['batch_size'].append(X_train.shape[2])

    lstm_model = KerasClassifier(create_model, verbose=0)

    lstm = GridSearchCV(lstm_model, parameters, scoring='accuracy', cv=cross_valid,
                       verbose=verbose_level).fit(np.append(X_train, X_valid, axis=0),np.append(y_train, y_valid))

    result = (lstm.best_score_, nr, lstm.best_params_)

    if result[0] > own_gyro[0]:
        own_gyro = result

1
Fitting 1 folds for each of 9 candidates, totalling 9 fits
[CV] batch_size=1, epochs=10, n_classes=6, num_units=15, optimizer=Adam, shape=15 
[CV]  batch_size=1, epochs=10, n_classes=6, num_units=15, optimizer=Adam, shape=15, score=0.5, total=  28.3s
[CV] batch_size=1, epochs=100, n_classes=6, num_units=15, optimizer=Adam, shape=15 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   28.7s remaining:    0.0s


[CV]  batch_size=1, epochs=100, n_classes=6, num_units=15, optimizer=Adam, shape=15, score=0.5769230769230769, total= 3.0min
[CV] batch_size=1, epochs=300, n_classes=6, num_units=15, optimizer=Adam, shape=15 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  3.5min remaining:    0.0s


[CV]  batch_size=1, epochs=300, n_classes=6, num_units=15, optimizer=Adam, shape=15, score=0.5961538461538461, total= 8.6min
[CV] batch_size=10, epochs=10, n_classes=6, num_units=15, optimizer=Adam, shape=15 
[CV]  batch_size=10, epochs=10, n_classes=6, num_units=15, optimizer=Adam, shape=15, score=0.38461538461538464, total=  13.4s
[CV] batch_size=10, epochs=100, n_classes=6, num_units=15, optimizer=Adam, shape=15 
[CV]  batch_size=10, epochs=100, n_classes=6, num_units=15, optimizer=Adam, shape=15, score=0.5769230769230769, total=  28.7s
[CV] batch_size=10, epochs=300, n_classes=6, num_units=15, optimizer=Adam, shape=15 
[CV]  batch_size=10, epochs=300, n_classes=6, num_units=15, optimizer=Adam, shape=15, score=0.5384615384615384, total= 1.3min
[CV] batch_size=100, epochs=10, n_classes=6, num_units=15, optimizer=Adam, shape=15 
[CV]  batch_size=100, epochs=10, n_classes=6, num_units=15, optimizer=Adam, shape=15, score=0.1346153846153846, total=  13.2s
[CV] batch_size=100, epochs=100,

...

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   49.4s remaining:    0.0s


[CV]  batch_size=1, epochs=100, n_classes=6, num_units=180, optimizer=Adam, shape=180, score=0.8653846153846154, total= 4.3min
[CV] batch_size=1, epochs=300, n_classes=6, num_units=180, optimizer=Adam, shape=180 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  5.2min remaining:    0.0s


[CV]  batch_size=1, epochs=300, n_classes=6, num_units=180, optimizer=Adam, shape=180, score=0.9038461538461539, total=12.0min
[CV] batch_size=10, epochs=10, n_classes=6, num_units=180, optimizer=Adam, shape=180 
[CV]  batch_size=10, epochs=10, n_classes=6, num_units=180, optimizer=Adam, shape=180, score=0.7307692307692307, total=  28.1s
[CV] batch_size=10, epochs=100, n_classes=6, num_units=180, optimizer=Adam, shape=180 
[CV]  batch_size=10, epochs=100, n_classes=6, num_units=180, optimizer=Adam, shape=180, score=0.8653846153846154, total=  49.5s
[CV] batch_size=10, epochs=300, n_classes=6, num_units=180, optimizer=Adam, shape=180 
[CV]  batch_size=10, epochs=300, n_classes=6, num_units=180, optimizer=Adam, shape=180, score=0.8846153846153846, total= 1.7min
[CV] batch_size=100, epochs=10, n_classes=6, num_units=180, optimizer=Adam, shape=180 
[CV]  batch_size=100, epochs=10, n_classes=6, num_units=180, optimizer=Adam, shape=180, score=0.5192307692307693, total=  26.0s
[CV] batch_size

[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed: 21.7min finished


In [22]:
print(own_gyro, end='\n')

(0.92307692307692313, 10, {'batch_size': 1, 'epochs': 100, 'n_classes': 6, 'num_units': 150, 'optimizer': 'Adam', 'shape': 150})


### Magnetometer

In [13]:
start_time = time.time()

for nr in frame_numbers:
    print(nr)
    try:
        ori_train_input, ori_train_label = prepare_data(train, nr, 'orientation', verbose=False)
        ori_valid_input, ori_valid_label = prepare_data(valid, nr, 'orientation', verbose=False)
    
        encoder = LabelEncoder()

        X_train = ori_train_input.reshape(ori_train_input.shape[0], 1, ori_train_input.shape[1])
        y_train = encoder.fit_transform(ori_train_label)
        X_valid = ori_valid_input.reshape(ori_valid_input.shape[0], 1, ori_valid_input.shape[1])
        y_valid = encoder.fit_transform(ori_valid_label)

        parameters[0]['n_classes'] = [len(encoder.classes_)]
        parameters[0]['shape'] = [X_train.shape[2]]
        parameters[0]['num_units'] = [X_train.shape[2]]

        parameters[0]['batch_size'] = [1, 10, 100]
        #parameters[0]['batch_size'].append(X_train.shape[2])

        lstm_model = KerasClassifier(create_model, verbose=0)

        lstm = GridSearchCV(lstm_model, parameters, scoring='accuracy', cv=cross_valid,
                           verbose=verbose_level).fit(np.append(X_train, X_valid, axis=0),np.append(y_train, y_valid))

        result = (lstm.best_score_, nr, lstm.best_params_)

        if result[0] > own_ori[0]:
            own_ori = result

        K.clear_session()
    
    except ValueError as err:
        pass

print('Duration: {}'.format(time.time() - start_time))

1
Fitting 1 folds for each of 9 candidates, totalling 9 fits
[CV] batch_size=1, epochs=10, n_classes=6, num_units=22, optimizer=Adam, shape=22 
[CV]  batch_size=1, epochs=10, n_classes=6, num_units=22, optimizer=Adam, shape=22, score=0.3269230769230769, total=  18.5s
[CV] batch_size=1, epochs=100, n_classes=6, num_units=22, optimizer=Adam, shape=22 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   18.7s remaining:    0.0s


[CV]  batch_size=1, epochs=100, n_classes=6, num_units=22, optimizer=Adam, shape=22, score=0.6730769230769231, total= 2.8min
[CV] batch_size=1, epochs=300, n_classes=6, num_units=22, optimizer=Adam, shape=22 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  3.1min remaining:    0.0s


[CV]  batch_size=1, epochs=300, n_classes=6, num_units=22, optimizer=Adam, shape=22, score=0.75, total= 7.8min
[CV] batch_size=10, epochs=10, n_classes=6, num_units=22, optimizer=Adam, shape=22 
[CV]  batch_size=10, epochs=10, n_classes=6, num_units=22, optimizer=Adam, shape=22, score=0.2692307692307692, total=   5.8s
[CV] batch_size=10, epochs=100, n_classes=6, num_units=22, optimizer=Adam, shape=22 
[CV]  batch_size=10, epochs=100, n_classes=6, num_units=22, optimizer=Adam, shape=22, score=0.5769230769230769, total=  15.7s
[CV] batch_size=10, epochs=300, n_classes=6, num_units=22, optimizer=Adam, shape=22 
[CV]  batch_size=10, epochs=300, n_classes=6, num_units=22, optimizer=Adam, shape=22, score=0.6923076923076923, total=  42.8s
[CV] batch_size=100, epochs=10, n_classes=6, num_units=22, optimizer=Adam, shape=22 
[CV]  batch_size=100, epochs=10, n_classes=6, num_units=22, optimizer=Adam, shape=22, score=0.1346153846153846, total=   3.2s
[CV] batch_size=100, epochs=100, n_classes=6, n

...

[CV]  batch_size=1, epochs=300, n_classes=6, num_units=132, optimizer=Adam, shape=132, score=0.75, total= 7.5min
[CV] batch_size=10, epochs=10, n_classes=6, num_units=132, optimizer=Adam, shape=132 
[CV]  batch_size=10, epochs=10, n_classes=6, num_units=132, optimizer=Adam, shape=132, score=0.46153846153846156, total=   3.6s
[CV] batch_size=10, epochs=100, n_classes=6, num_units=132, optimizer=Adam, shape=132 
[CV]  batch_size=10, epochs=100, n_classes=6, num_units=132, optimizer=Adam, shape=132, score=0.7307692307692307, total=  16.3s
[CV] batch_size=10, epochs=300, n_classes=6, num_units=132, optimizer=Adam, shape=132 
[CV]  batch_size=10, epochs=300, n_classes=6, num_units=132, optimizer=Adam, shape=132, score=0.75, total=  43.7s
[CV] batch_size=100, epochs=10, n_classes=6, num_units=132, optimizer=Adam, shape=132 
[CV]  batch_size=100, epochs=10, n_classes=6, num_units=132, optimizer=Adam, shape=132, score=0.3076923076923077, total=   2.7s
[CV] batch_size=100, epochs=100, n_classes

[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed: 11.7min finished


7


c:\python36\lib\site-packages\scipy\stats\stats.py:3021: RuntimeWarning: invalid value encountered in double_scalars
  r = r_num / r_den
c:\python36\lib\site-packages\scipy\stats\stats.py:5277: RuntimeWarning: invalid value encountered in less
  x = np.where(x < 1.0, x, 1.0)  # if x > 1 then return 1.0


8
9
10
11
12
Duration: 6219.480236053467


In [14]:
print(own_ori, end='\n')

(0.90384615384615385, 2, {'batch_size': 1, 'epochs': 300, 'n_classes': 6, 'num_units': 44, 'optimizer': 'Adam', 'shape': 44})


### EMG

In [18]:
start_time = time.time()

for nr in frame_numbers:
    print(nr)
    try:
        emg_train_input, emg_train_label = prepare_data(train, nr, 'emg', verbose=False)
        emg_valid_input, emg_valid_label = prepare_data(valid, nr, 'emg', verbose=False)

        encoder = LabelEncoder()

        X_train = emg_train_input.reshape(emg_train_input.shape[0], 1, emg_train_input.shape[1])
        y_train = encoder.fit_transform(emg_train_label)
        X_valid = emg_valid_input.reshape(emg_valid_input.shape[0], 1, emg_valid_input.shape[1])
        y_valid = encoder.fit_transform(emg_valid_label)

        parameters[0]['n_classes'] = [len(encoder.classes_)]
        parameters[0]['shape'] = [X_train.shape[2]]
        parameters[0]['num_units'] = [X_train.shape[2]]

        #parameters[0]['batch_size'] = [1, 10, 100]
        #parameters[0]['batch_size'].append(X_train.shape[2])

        lstm_model = KerasClassifier(create_model, verbose=0)

        lstm = GridSearchCV(lstm_model, parameters, scoring='accuracy', cv=cross_valid,
                           verbose=verbose_level).fit(np.append(X_train, X_valid, axis=0),np.append(y_train, y_valid))

        result = (lstm.best_score_, nr, lstm.best_params_)

        if result[0] > own_emg[0]:
            own_emg = result

        K.clear_session()
    
    except ValueError as err:
        pass
    
print('Duration: {}'.format(time.time() - start_time))

1
Fitting 1 folds for each of 9 candidates, totalling 9 fits
[CV] batch_size=1, epochs=10, n_classes=6, num_units=32, optimizer=Adam, shape=32 
[CV]  batch_size=1, epochs=10, n_classes=6, num_units=32, optimizer=Adam, shape=32, score=0.5, total=  21.7s
[CV] batch_size=1, epochs=100, n_classes=6, num_units=32, optimizer=Adam, shape=32 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   22.0s remaining:    0.0s


[CV]  batch_size=1, epochs=100, n_classes=6, num_units=32, optimizer=Adam, shape=32, score=0.5384615384615384, total= 2.9min
[CV] batch_size=1, epochs=300, n_classes=6, num_units=32, optimizer=Adam, shape=32 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  3.3min remaining:    0.0s


[CV]  batch_size=1, epochs=300, n_classes=6, num_units=32, optimizer=Adam, shape=32, score=0.6923076923076923, total= 9.0min
[CV] batch_size=10, epochs=10, n_classes=6, num_units=32, optimizer=Adam, shape=32 
[CV]  batch_size=10, epochs=10, n_classes=6, num_units=32, optimizer=Adam, shape=32, score=0.28846153846153844, total=   4.7s
[CV] batch_size=10, epochs=100, n_classes=6, num_units=32, optimizer=Adam, shape=32 
[CV]  batch_size=10, epochs=100, n_classes=6, num_units=32, optimizer=Adam, shape=32, score=0.5384615384615384, total=  20.5s
[CV] batch_size=10, epochs=300, n_classes=6, num_units=32, optimizer=Adam, shape=32 
[CV]  batch_size=10, epochs=300, n_classes=6, num_units=32, optimizer=Adam, shape=32, score=0.5961538461538461, total=  53.4s
[CV] batch_size=100, epochs=10, n_classes=6, num_units=32, optimizer=Adam, shape=32 
[CV]  batch_size=100, epochs=10, n_classes=6, num_units=32, optimizer=Adam, shape=32, score=0.28846153846153844, total=   3.5s
[CV] batch_size=100, epochs=100

...

[CV]  batch_size=1, epochs=100, n_classes=6, num_units=384, optimizer=Adam, shape=384, score=0.7884615384615384, total= 3.2min
[CV] batch_size=1, epochs=300, n_classes=6, num_units=384, optimizer=Adam, shape=384 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  3.6min remaining:    0.0s


[CV]  batch_size=1, epochs=300, n_classes=6, num_units=384, optimizer=Adam, shape=384, score=0.8461538461538461, total= 9.6min
[CV] batch_size=10, epochs=10, n_classes=6, num_units=384, optimizer=Adam, shape=384 
[CV]  batch_size=10, epochs=10, n_classes=6, num_units=384, optimizer=Adam, shape=384, score=0.6538461538461539, total=   4.5s
[CV] batch_size=10, epochs=100, n_classes=6, num_units=384, optimizer=Adam, shape=384 
[CV]  batch_size=10, epochs=100, n_classes=6, num_units=384, optimizer=Adam, shape=384, score=0.7692307692307693, total=  24.2s
[CV] batch_size=10, epochs=300, n_classes=6, num_units=384, optimizer=Adam, shape=384 
[CV]  batch_size=10, epochs=300, n_classes=6, num_units=384, optimizer=Adam, shape=384, score=0.7692307692307693, total= 1.1min
[CV] batch_size=100, epochs=10, n_classes=6, num_units=384, optimizer=Adam, shape=384 
[CV]  batch_size=100, epochs=10, n_classes=6, num_units=384, optimizer=Adam, shape=384, score=0.23076923076923078, total=   3.0s
[CV] batch_siz

[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed: 15.2min finished


Duration: 15771.902138471603


In [19]:
print(own_emg, end='\n')

(0.84615384615384615, 10, {'batch_size': 1, 'epochs': 300, 'n_classes': 6, 'num_units': 320, 'optimizer': 'Adam', 'shape': 320})


## Parametertest - paper

In [6]:
train, valid, test = load_data_for_classification('paper')

# lists will be filled with (accuracy, frame_number, params)
paper_acc = (0, 0)
paper_gyro = (0, 0)
paper_ori = (0, 0)
paper_emg = (0, 0)

### Accelerometer

In [7]:
for nr in frame_numbers:
    print(nr)
    try:
        acc_train_input, acc_train_label = prepare_data(train, nr, 'accelerometer', verbose=False)
        acc_valid_input, acc_valid_label = prepare_data(valid, nr, 'accelerometer', verbose=False)

        encoder = LabelEncoder()

        X_train = acc_train_input.reshape(acc_train_input.shape[0], 1, acc_train_input.shape[1])
        y_train = encoder.fit_transform(acc_train_label)
        X_valid = acc_valid_input.reshape(acc_valid_input.shape[0], 1, acc_valid_input.shape[1])
        y_valid = encoder.fit_transform(acc_valid_label)

        parameters[0]['n_classes'] = [len(encoder.classes_)]
        parameters[0]['shape'] = [X_train.shape[2]]
        parameters[0]['num_units'] = [X_train.shape[2]]

        #parameters[0]['batch_size'] = [1, 10, 100]
        #parameters[0]['batch_size'].append(X_train.shape[2])

        lstm_model = KerasClassifier(create_model, verbose=0)

        lstm = GridSearchCV(lstm_model, parameters, scoring='accuracy', cv=cross_valid,
                           verbose=verbose_level).fit(np.append(X_train, X_valid, axis=0),np.append(y_train, y_valid))

        result = (lstm.best_score_, nr, lstm.best_params_)

        if result[0] > paper_acc[0]:
            paper_acc = result

        K.clear_session()
    
    except ValueError as err:
        pass

1
Fitting 1 folds for each of 9 candidates, totalling 9 fits
[CV] batch_size=1, epochs=10, n_classes=18, num_units=15, optimizer=Adam, shape=15 
[CV]  batch_size=1, epochs=10, n_classes=18, num_units=15, optimizer=Adam, shape=15, score=0.08974358974358974, total=  53.4s
[CV] batch_size=1, epochs=100, n_classes=18, num_units=15, optimizer=Adam, shape=15 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   54.2s remaining:    0.0s


[CV]  batch_size=1, epochs=100, n_classes=18, num_units=15, optimizer=Adam, shape=15, score=0.21794871794871795, total= 8.5min
[CV] batch_size=1, epochs=300, n_classes=18, num_units=15, optimizer=Adam, shape=15 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  9.4min remaining:    0.0s


[CV]  batch_size=1, epochs=300, n_classes=18, num_units=15, optimizer=Adam, shape=15, score=0.23717948717948717, total=37.3min
[CV] batch_size=10, epochs=10, n_classes=18, num_units=15, optimizer=Adam, shape=15 
[CV]  batch_size=10, epochs=10, n_classes=18, num_units=15, optimizer=Adam, shape=15, score=0.0641025641025641, total=   9.0s
[CV] batch_size=10, epochs=100, n_classes=18, num_units=15, optimizer=Adam, shape=15 
[CV]  batch_size=10, epochs=100, n_classes=18, num_units=15, optimizer=Adam, shape=15, score=0.1346153846153846, total=  51.9s
[CV] batch_size=10, epochs=300, n_classes=18, num_units=15, optimizer=Adam, shape=15 
[CV]  batch_size=10, epochs=300, n_classes=18, num_units=15, optimizer=Adam, shape=15, score=0.21794871794871795, total= 2.5min
[CV] batch_size=100, epochs=10, n_classes=18, num_units=15, optimizer=Adam, shape=15 
[CV]  batch_size=100, epochs=10, n_classes=18, num_units=15, optimizer=Adam, shape=15, score=0.03205128205128205, total=   3.5s
[CV] batch_size=100, 

...

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   43.2s remaining:    0.0s


[CV]  batch_size=1, epochs=100, n_classes=18, num_units=180, optimizer=Adam, shape=180, score=0.38064516129032255, total= 6.8min
[CV] batch_size=1, epochs=300, n_classes=18, num_units=180, optimizer=Adam, shape=180 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  7.5min remaining:    0.0s


[CV]  batch_size=1, epochs=300, n_classes=18, num_units=180, optimizer=Adam, shape=180, score=0.33548387096774196, total=21.7min
[CV] batch_size=10, epochs=10, n_classes=18, num_units=180, optimizer=Adam, shape=180 
[CV]  batch_size=10, epochs=10, n_classes=18, num_units=180, optimizer=Adam, shape=180, score=0.24516129032258063, total=   7.3s
[CV] batch_size=10, epochs=100, n_classes=18, num_units=180, optimizer=Adam, shape=180 
[CV]  batch_size=10, epochs=100, n_classes=18, num_units=180, optimizer=Adam, shape=180, score=0.3935483870967742, total=  53.0s
[CV] batch_size=10, epochs=300, n_classes=18, num_units=180, optimizer=Adam, shape=180 
[CV]  batch_size=10, epochs=300, n_classes=18, num_units=180, optimizer=Adam, shape=180, score=0.3548387096774194, total= 2.8min
[CV] batch_size=100, epochs=10, n_classes=18, num_units=180, optimizer=Adam, shape=180 
[CV]  batch_size=100, epochs=10, n_classes=18, num_units=180, optimizer=Adam, shape=180, score=0.1032258064516129, total=   2.9s
[CV]

[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed: 33.7min finished


In [8]:
print(paper_acc, end='\n')

(0.59615384615384615, 5, {'batch_size': 1, 'epochs': 300, 'n_classes': 18, 'num_units': 75, 'optimizer': 'Adam', 'shape': 75})


### Gyroskop

In [8]:
for nr in frame_numbers:
    print(nr)
    try:
        gyro_train_input, gyro_train_label = prepare_data(train, nr, 'gyro', verbose=False)
        gyro_valid_input, gyro_valid_label = prepare_data(valid, nr, 'gyro', verbose=False)

        encoder = LabelEncoder()

        X_train = gyro_train_input.reshape(gyro_train_input.shape[0], 1, gyro_train_input.shape[1])
        y_train = encoder.fit_transform(gyro_train_label)
        X_valid = gyro_valid_input.reshape(gyro_valid_input.shape[0], 1, gyro_valid_input.shape[1])
        y_valid = encoder.fit_transform(gyro_valid_label)

        parameters[0]['n_classes'] = [len(encoder.classes_)]
        parameters[0]['shape'] = [X_train.shape[2]]
        parameters[0]['num_units'] = [X_train.shape[2]]

        #parameters[0]['batch_size'] = [1, 10, 100]
        #parameters[0]['batch_size'].append(X_train.shape[2])

        lstm_model = KerasClassifier(create_model, verbose=0)

        lstm = GridSearchCV(lstm_model, parameters, scoring='accuracy', cv=cross_valid,
                           verbose=verbose_level).fit(np.append(X_train, X_valid, axis=0),np.append(y_train, y_valid))

        result = (lstm.best_score_, nr, lstm.best_params_)

        if result[0] > paper_gyro[0]:
            paper_gyro = result

        K.clear_session()
    
    except ValueError as err:
        pass

1
Fitting 1 folds for each of 9 candidates, totalling 9 fits
[CV] batch_size=1, epochs=10, n_classes=18, num_units=15, optimizer=Adam, shape=15 
[CV]  batch_size=1, epochs=10, n_classes=18, num_units=15, optimizer=Adam, shape=15, score=0.1282051282051282, total=  12.6s
[CV] batch_size=1, epochs=100, n_classes=18, num_units=15, optimizer=Adam, shape=15 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   12.8s remaining:    0.0s


[CV]  batch_size=1, epochs=100, n_classes=18, num_units=15, optimizer=Adam, shape=15, score=0.11538461538461539, total= 1.8min
[CV] batch_size=1, epochs=300, n_classes=18, num_units=15, optimizer=Adam, shape=15 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  2.0min remaining:    0.0s


[CV]  batch_size=1, epochs=300, n_classes=18, num_units=15, optimizer=Adam, shape=15, score=0.16025641025641027, total= 5.4min
[CV] batch_size=10, epochs=10, n_classes=18, num_units=15, optimizer=Adam, shape=15 
[CV]  batch_size=10, epochs=10, n_classes=18, num_units=15, optimizer=Adam, shape=15, score=0.14102564102564102, total=   3.7s
[CV] batch_size=10, epochs=100, n_classes=18, num_units=15, optimizer=Adam, shape=15 
[CV]  batch_size=10, epochs=100, n_classes=18, num_units=15, optimizer=Adam, shape=15, score=0.1346153846153846, total=  13.4s
[CV] batch_size=10, epochs=300, n_classes=18, num_units=15, optimizer=Adam, shape=15 
[CV]  batch_size=10, epochs=300, n_classes=18, num_units=15, optimizer=Adam, shape=15, score=0.17307692307692307, total=  35.8s
[CV] batch_size=100, epochs=10, n_classes=18, num_units=15, optimizer=Adam, shape=15 
[CV]  batch_size=100, epochs=10, n_classes=18, num_units=15, optimizer=Adam, shape=15, score=0.05128205128205128, total=   2.8s
[CV] batch_size=100,

...

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   34.9s remaining:    0.0s


[CV]  batch_size=1, epochs=100, n_classes=18, num_units=180, optimizer=Adam, shape=180, score=0.4064516129032258, total= 5.4min
[CV] batch_size=1, epochs=300, n_classes=18, num_units=180, optimizer=Adam, shape=180 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  6.0min remaining:    0.0s


[CV]  batch_size=1, epochs=300, n_classes=18, num_units=180, optimizer=Adam, shape=180, score=0.4258064516129032, total=16.9min
[CV] batch_size=10, epochs=10, n_classes=18, num_units=180, optimizer=Adam, shape=180 
[CV]  batch_size=10, epochs=10, n_classes=18, num_units=180, optimizer=Adam, shape=180, score=0.24516129032258063, total=   8.7s
[CV] batch_size=10, epochs=100, n_classes=18, num_units=180, optimizer=Adam, shape=180 
[CV]  batch_size=10, epochs=100, n_classes=18, num_units=180, optimizer=Adam, shape=180, score=0.34838709677419355, total=  35.5s
[CV] batch_size=10, epochs=300, n_classes=18, num_units=180, optimizer=Adam, shape=180 
[CV]  batch_size=10, epochs=300, n_classes=18, num_units=180, optimizer=Adam, shape=180, score=0.3870967741935484, total= 1.8min
[CV] batch_size=100, epochs=10, n_classes=18, num_units=180, optimizer=Adam, shape=180 
[CV]  batch_size=100, epochs=10, n_classes=18, num_units=180, optimizer=Adam, shape=180, score=0.14838709677419354, total=   3.0s
[CV

[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed: 26.0min finished


In [9]:
print(paper_gyro, end='\n')

(0.52564102564102566, 10, {'batch_size': 1, 'epochs': 300, 'n_classes': 18, 'num_units': 150, 'optimizer': 'Adam', 'shape': 150})


### Magnetometer

In [10]:
for nr in frame_numbers:
    print(nr)
    try:
        ori_train_input, ori_train_label = prepare_data(train, nr, 'orientation', verbose=False)
        ori_valid_input, ori_valid_label = prepare_data(valid, nr, 'orientation', verbose=False)

        encoder = LabelEncoder()

        X_train = ori_train_input.reshape(ori_train_input.shape[0], 1, ori_train_input.shape[1])
        y_train = encoder.fit_transform(ori_train_label)
        X_valid = ori_valid_input.reshape(ori_valid_input.shape[0], 1, ori_valid_input.shape[1])
        y_valid = encoder.fit_transform(ori_valid_label)

        parameters[0]['n_classes'] = [len(encoder.classes_)]
        parameters[0]['shape'] = [X_train.shape[2]]
        parameters[0]['num_units'] = [X_train.shape[2]]

        #parameters[0]['batch_size'] = [1, 10, 100]
        #parameters[0]['batch_size'].append(X_train.shape[2])

        lstm_model = KerasClassifier(create_model, verbose=0)

        lstm = GridSearchCV(lstm_model, parameters, scoring='accuracy', cv=cross_valid,
                           verbose=verbose_level).fit(np.append(X_train, X_valid, axis=0),np.append(y_train, y_valid))

        result = (lstm.best_score_, nr, lstm.best_params_)

        if result[0] > paper_ori[0]:
            paper_ori = result

        K.clear_session()
    
    except ValueError as err:
        pass

1
Fitting 1 folds for each of 9 candidates, totalling 9 fits
[CV] batch_size=1, epochs=10, n_classes=18, num_units=22, optimizer=Adam, shape=22 
[CV]  batch_size=1, epochs=10, n_classes=18, num_units=22, optimizer=Adam, shape=22, score=0.07692307692307693, total=  12.8s
[CV] batch_size=1, epochs=100, n_classes=18, num_units=22, optimizer=Adam, shape=22 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   13.0s remaining:    0.0s


[CV]  batch_size=1, epochs=100, n_classes=18, num_units=22, optimizer=Adam, shape=22, score=0.23717948717948717, total= 1.8min
[CV] batch_size=1, epochs=300, n_classes=18, num_units=22, optimizer=Adam, shape=22 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  2.0min remaining:    0.0s


[CV]  batch_size=1, epochs=300, n_classes=18, num_units=22, optimizer=Adam, shape=22, score=0.3076923076923077, total= 5.3min
[CV] batch_size=10, epochs=10, n_classes=18, num_units=22, optimizer=Adam, shape=22 
[CV]  batch_size=10, epochs=10, n_classes=18, num_units=22, optimizer=Adam, shape=22, score=0.03205128205128205, total=   3.3s
[CV] batch_size=10, epochs=100, n_classes=18, num_units=22, optimizer=Adam, shape=22 
[CV]  batch_size=10, epochs=100, n_classes=18, num_units=22, optimizer=Adam, shape=22, score=0.17307692307692307, total=  13.2s
[CV] batch_size=10, epochs=300, n_classes=18, num_units=22, optimizer=Adam, shape=22 
[CV]  batch_size=10, epochs=300, n_classes=18, num_units=22, optimizer=Adam, shape=22, score=0.2564102564102564, total=  38.2s
[CV] batch_size=100, epochs=10, n_classes=18, num_units=22, optimizer=Adam, shape=22 
[CV]  batch_size=100, epochs=10, n_classes=18, num_units=22, optimizer=Adam, shape=22, score=0.03205128205128205, total=   2.4s
[CV] batch_size=100, 

...

[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed: 10.9min remaining:    0.0s


[CV]  batch_size=1, epochs=300, n_classes=18, num_units=242, optimizer=Adam, shape=242, score=0.23076923076923078, total=27.5min
[CV] batch_size=10, epochs=10, n_classes=18, num_units=242, optimizer=Adam, shape=242 
[CV]  batch_size=10, epochs=10, n_classes=18, num_units=242, optimizer=Adam, shape=242, score=0.10256410256410256, total=   7.9s
[CV] batch_size=10, epochs=100, n_classes=18, num_units=242, optimizer=Adam, shape=242 
[CV]  batch_size=10, epochs=100, n_classes=18, num_units=242, optimizer=Adam, shape=242, score=0.22435897435897437, total=  57.9s
[CV] batch_size=10, epochs=300, n_classes=18, num_units=242, optimizer=Adam, shape=242 
[CV]  batch_size=10, epochs=300, n_classes=18, num_units=242, optimizer=Adam, shape=242, score=0.24358974358974358, total= 2.8min
[CV] batch_size=100, epochs=10, n_classes=18, num_units=242, optimizer=Adam, shape=242 
[CV]  batch_size=100, epochs=10, n_classes=18, num_units=242, optimizer=Adam, shape=242, score=0.05128205128205128, total=   3.5s
[

[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed: 43.1min finished


12


c:\python36\lib\site-packages\scipy\stats\stats.py:3021: RuntimeWarning: invalid value encountered in double_scalars
  r = r_num / r_den
c:\python36\lib\site-packages\scipy\stats\stats.py:5277: RuntimeWarning: invalid value encountered in less
  x = np.where(x < 1.0, x, 1.0)  # if x > 1 then return 1.0


In [11]:
print(paper_ori, end='\n')

(0.57692307692307687, 3, {'batch_size': 1, 'epochs': 300, 'n_classes': 18, 'num_units': 66, 'optimizer': 'Adam', 'shape': 66})


### EMG

In [12]:
for nr in frame_numbers:
    print(nr)
    try:
        emg_train_input, emg_train_label = prepare_data(train, nr, 'emg', verbose=False)
        emg_valid_input, emg_valid_label = prepare_data(valid, nr, 'emg', verbose=False)

        encoder = LabelEncoder()

        X_train = emg_train_input.reshape(emg_train_input.shape[0], 1, emg_train_input.shape[1])
        y_train = encoder.fit_transform(emg_train_label)
        X_valid = emg_valid_input.reshape(emg_valid_input.shape[0], 1, emg_valid_input.shape[1])
        y_valid = encoder.fit_transform(emg_valid_label)

        parameters[0]['n_classes'] = [len(encoder.classes_)]
        parameters[0]['shape'] = [X_train.shape[2]]
        parameters[0]['num_units'] = [X_train.shape[2]]

        #parameters[0]['batch_size'] = [1, 10, 100]
        #parameters[0]['batch_size'].append(X_train.shape[2])

        lstm_model = KerasClassifier(create_model, verbose=0)

        lstm = GridSearchCV(lstm_model, parameters, scoring='accuracy', cv=cross_valid,
                           verbose=verbose_level).fit(np.append(X_train, X_valid, axis=0),np.append(y_train, y_valid))

        result = (lstm.best_score_, nr, lstm.best_params_)

        if result[0] > paper_emg[0]:
            paper_emg = result

        K.clear_session()
    
    except ValueError as err:
        pass

1
Fitting 1 folds for each of 9 candidates, totalling 9 fits
[CV] batch_size=1, epochs=10, n_classes=18, num_units=32, optimizer=Adam, shape=32 
[CV]  batch_size=1, epochs=10, n_classes=18, num_units=32, optimizer=Adam, shape=32, score=0.12179487179487179, total=  13.3s
[CV] batch_size=1, epochs=100, n_classes=18, num_units=32, optimizer=Adam, shape=32 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   13.6s remaining:    0.0s


[CV]  batch_size=1, epochs=100, n_classes=18, num_units=32, optimizer=Adam, shape=32, score=0.08333333333333333, total= 1.9min
[CV] batch_size=1, epochs=300, n_classes=18, num_units=32, optimizer=Adam, shape=32 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  2.1min remaining:    0.0s


[CV]  batch_size=1, epochs=300, n_classes=18, num_units=32, optimizer=Adam, shape=32, score=0.15384615384615385, total= 5.6min
[CV] batch_size=10, epochs=10, n_classes=18, num_units=32, optimizer=Adam, shape=32 
[CV]  batch_size=10, epochs=10, n_classes=18, num_units=32, optimizer=Adam, shape=32, score=0.0641025641025641, total=   3.5s
[CV] batch_size=10, epochs=100, n_classes=18, num_units=32, optimizer=Adam, shape=32 
[CV]  batch_size=10, epochs=100, n_classes=18, num_units=32, optimizer=Adam, shape=32, score=0.11538461538461539, total=  15.9s
[CV] batch_size=10, epochs=300, n_classes=18, num_units=32, optimizer=Adam, shape=32 
[CV]  batch_size=10, epochs=300, n_classes=18, num_units=32, optimizer=Adam, shape=32, score=0.1282051282051282, total=  37.2s
[CV] batch_size=100, epochs=10, n_classes=18, num_units=32, optimizer=Adam, shape=32 
[CV]  batch_size=100, epochs=10, n_classes=18, num_units=32, optimizer=Adam, shape=32, score=0.03205128205128205, total=   2.6s
[CV] batch_size=100, 

...

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  2.9min remaining:    0.0s


[CV]  batch_size=1, epochs=100, n_classes=18, num_units=384, optimizer=Adam, shape=384, score=0.358974358974359, total=28.5min
[CV] batch_size=1, epochs=300, n_classes=18, num_units=384, optimizer=Adam, shape=384 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed: 31.4min remaining:    0.0s


[CV]  batch_size=1, epochs=300, n_classes=18, num_units=384, optimizer=Adam, shape=384, score=0.358974358974359, total=86.3min
[CV] batch_size=10, epochs=10, n_classes=18, num_units=384, optimizer=Adam, shape=384 
[CV]  batch_size=10, epochs=10, n_classes=18, num_units=384, optimizer=Adam, shape=384, score=0.19230769230769232, total=  22.3s
[CV] batch_size=10, epochs=100, n_classes=18, num_units=384, optimizer=Adam, shape=384 
[CV]  batch_size=10, epochs=100, n_classes=18, num_units=384, optimizer=Adam, shape=384, score=0.3333333333333333, total= 3.3min
[CV] batch_size=10, epochs=300, n_classes=18, num_units=384, optimizer=Adam, shape=384 
[CV]  batch_size=10, epochs=300, n_classes=18, num_units=384, optimizer=Adam, shape=384, score=0.34615384615384615, total= 9.5min
[CV] batch_size=100, epochs=10, n_classes=18, num_units=384, optimizer=Adam, shape=384 
[CV]  batch_size=100, epochs=10, n_classes=18, num_units=384, optimizer=Adam, shape=384, score=0.14102564102564102, total=   5.1s
[CV]

[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed: 132.8min finished


In [13]:
print(paper_emg, end='\n')

(0.46153846153846156, 10, {'batch_size': 1, 'epochs': 100, 'n_classes': 18, 'num_units': 320, 'optimizer': 'Adam', 'shape': 320})


## Parametertest - all

In [6]:
train, valid, test = load_data_for_classification('all')

# lists will be filled with (accuracy, frame_number, params)
all_acc = (0, 0)
all_gyro = (0, 0)
all_ori = (0, 0)
all_emg = (0, 0)

### Accelerometer

In [7]:
for nr in frame_numbers:
    print(nr)
    try:
        acc_train_input, acc_train_label = prepare_data(train, nr, 'accelerometer', verbose=False)
        acc_valid_input, acc_valid_label = prepare_data(valid, nr, 'accelerometer', verbose=False)

        encoder = LabelEncoder()

        X_train = acc_train_input.reshape(acc_train_input.shape[0], 1, acc_train_input.shape[1])
        y_train = encoder.fit_transform(acc_train_label)
        X_valid = acc_valid_input.reshape(acc_valid_input.shape[0], 1, acc_valid_input.shape[1])
        y_valid = encoder.fit_transform(acc_valid_label)

        parameters[0]['n_classes'] = [len(encoder.classes_)]
        parameters[0]['shape'] = [X_train.shape[2]]
        parameters[0]['num_units'] = [X_train.shape[2]]

        #parameters[0]['batch_size'] = [1, 10, 100]
        #parameters[0]['batch_size'].append(X_train.shape[2])

        lstm_model = KerasClassifier(create_model, verbose=0)

        lstm = GridSearchCV(lstm_model, parameters, scoring='accuracy', cv=cross_valid,
                           verbose=verbose_level).fit(np.append(X_train, X_valid, axis=0),np.append(y_train, y_valid))

        result = (lstm.best_score_, nr, lstm.best_params_)

        if result[0] > all_acc[0]:
            all_acc = result

        K.clear_session()
    
    except ValueError as err:
        pass

1
Fitting 1 folds for each of 9 candidates, totalling 9 fits
[CV] batch_size=1, epochs=10, n_classes=24, num_units=15, optimizer=Adam, shape=15 
[CV]  batch_size=1, epochs=10, n_classes=24, num_units=15, optimizer=Adam, shape=15, score=0.125, total=  17.1s
[CV] batch_size=1, epochs=100, n_classes=24, num_units=15, optimizer=Adam, shape=15 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   17.5s remaining:    0.0s


[CV]  batch_size=1, epochs=100, n_classes=24, num_units=15, optimizer=Adam, shape=15, score=0.27884615384615385, total= 2.4min
[CV] batch_size=1, epochs=300, n_classes=24, num_units=15, optimizer=Adam, shape=15 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  2.7min remaining:    0.0s


[CV]  batch_size=1, epochs=300, n_classes=24, num_units=15, optimizer=Adam, shape=15, score=0.30288461538461536, total= 7.4min
[CV] batch_size=10, epochs=10, n_classes=24, num_units=15, optimizer=Adam, shape=15 
[CV]  batch_size=10, epochs=10, n_classes=24, num_units=15, optimizer=Adam, shape=15, score=0.1201923076923077, total=   3.9s
[CV] batch_size=10, epochs=100, n_classes=24, num_units=15, optimizer=Adam, shape=15 
[CV]  batch_size=10, epochs=100, n_classes=24, num_units=15, optimizer=Adam, shape=15, score=0.1778846153846154, total=  16.8s
[CV] batch_size=10, epochs=300, n_classes=24, num_units=15, optimizer=Adam, shape=15 
[CV]  batch_size=10, epochs=300, n_classes=24, num_units=15, optimizer=Adam, shape=15, score=0.28846153846153844, total=  46.1s
[CV] batch_size=100, epochs=10, n_classes=24, num_units=15, optimizer=Adam, shape=15 
[CV]  batch_size=100, epochs=10, n_classes=24, num_units=15, optimizer=Adam, shape=15, score=0.04807692307692308, total=   2.9s
[CV] batch_size=100, 

...

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   55.6s remaining:    0.0s


[CV]  batch_size=1, epochs=100, n_classes=24, num_units=180, optimizer=Adam, shape=180, score=0.4927536231884058, total= 8.1min
[CV] batch_size=1, epochs=300, n_classes=24, num_units=180, optimizer=Adam, shape=180 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  9.1min remaining:    0.0s


[CV]  batch_size=1, epochs=300, n_classes=24, num_units=180, optimizer=Adam, shape=180, score=0.5458937198067633, total=25.7min
[CV] batch_size=10, epochs=10, n_classes=24, num_units=180, optimizer=Adam, shape=180 
[CV]  batch_size=10, epochs=10, n_classes=24, num_units=180, optimizer=Adam, shape=180, score=0.3333333333333333, total=   9.7s
[CV] batch_size=10, epochs=100, n_classes=24, num_units=180, optimizer=Adam, shape=180 
[CV]  batch_size=10, epochs=100, n_classes=24, num_units=180, optimizer=Adam, shape=180, score=0.45893719806763283, total=  58.9s
[CV] batch_size=10, epochs=300, n_classes=24, num_units=180, optimizer=Adam, shape=180 
[CV]  batch_size=10, epochs=300, n_classes=24, num_units=180, optimizer=Adam, shape=180, score=0.4927536231884058, total= 2.9min
[CV] batch_size=100, epochs=10, n_classes=24, num_units=180, optimizer=Adam, shape=180 
[CV]  batch_size=100, epochs=10, n_classes=24, num_units=180, optimizer=Adam, shape=180, score=0.15458937198067632, total=   5.6s
[CV]

[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed: 39.6min finished


In [8]:
print(all_acc, end='\n')

(0.64251207729468596, 4, {'batch_size': 1, 'epochs': 300, 'n_classes': 24, 'num_units': 60, 'optimizer': 'Adam', 'shape': 60})


### Gyroskop

In [9]:
for nr in frame_numbers:
    print(nr)
    try:
        gyro_train_input, gyro_train_label = prepare_data(train, nr, 'gyro', verbose=False)
        gyro_valid_input, gyro_valid_label = prepare_data(valid, nr, 'gyro', verbose=False)

        encoder = LabelEncoder()

        X_train = gyro_train_input.reshape(gyro_train_input.shape[0], 1, gyro_train_input.shape[1])
        y_train = encoder.fit_transform(gyro_train_label)
        X_valid = gyro_valid_input.reshape(gyro_valid_input.shape[0], 1, gyro_valid_input.shape[1])
        y_valid = encoder.fit_transform(gyro_valid_label)

        parameters[0]['n_classes'] = [len(encoder.classes_)]
        parameters[0]['shape'] = [X_train.shape[2]]
        parameters[0]['num_units'] = [X_train.shape[2]]

        #parameters[0]['batch_size'] = [1, 10, 100]
        #parameters[0]['batch_size'].append(X_train.shape[2])

        lstm_model = KerasClassifier(create_model, verbose=0)

        lstm = GridSearchCV(lstm_model, parameters, scoring='accuracy', cv=cross_valid,
                           verbose=verbose_level).fit(np.append(X_train, X_valid, axis=0),np.append(y_train, y_valid))

        result = (lstm.best_score_, nr, lstm.best_params_)

        if result[0] > all_gyro[0]:
            all_gyro = result

        K.clear_session()
    
    except ValueError as err:
        pass

1
Fitting 1 folds for each of 9 candidates, totalling 9 fits
[CV] batch_size=1, epochs=10, n_classes=24, num_units=15, optimizer=Adam, shape=15 
[CV]  batch_size=1, epochs=10, n_classes=24, num_units=15, optimizer=Adam, shape=15, score=0.17307692307692307, total=  22.4s
[CV] batch_size=1, epochs=100, n_classes=24, num_units=15, optimizer=Adam, shape=15 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   22.9s remaining:    0.0s


[CV]  batch_size=1, epochs=100, n_classes=24, num_units=15, optimizer=Adam, shape=15, score=0.21634615384615385, total= 3.2min
[CV] batch_size=1, epochs=300, n_classes=24, num_units=15, optimizer=Adam, shape=15 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  3.6min remaining:    0.0s


[CV]  batch_size=1, epochs=300, n_classes=24, num_units=15, optimizer=Adam, shape=15, score=0.21153846153846154, total= 9.9min
[CV] batch_size=10, epochs=10, n_classes=24, num_units=15, optimizer=Adam, shape=15 
[CV]  batch_size=10, epochs=10, n_classes=24, num_units=15, optimizer=Adam, shape=15, score=0.12980769230769232, total=   5.9s
[CV] batch_size=10, epochs=100, n_classes=24, num_units=15, optimizer=Adam, shape=15 
[CV]  batch_size=10, epochs=100, n_classes=24, num_units=15, optimizer=Adam, shape=15, score=0.16826923076923078, total=  25.6s
[CV] batch_size=10, epochs=300, n_classes=24, num_units=15, optimizer=Adam, shape=15 
[CV]  batch_size=10, epochs=300, n_classes=24, num_units=15, optimizer=Adam, shape=15, score=0.21153846153846154, total= 1.1min
[CV] batch_size=100, epochs=10, n_classes=24, num_units=15, optimizer=Adam, shape=15 
[CV]  batch_size=100, epochs=10, n_classes=24, num_units=15, optimizer=Adam, shape=15, score=0.0673076923076923, total=   4.8s
[CV] batch_size=100,

[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed: 15.6min finished


KeyboardInterrupt: 

In [ ]:
print(all_gyro, end='\n')

### Magnetometer

In [ ]:
for nr in frame_numbers:
    print(nr)
    try:
        ori_train_input, ori_train_label = prepare_data(train, nr, 'orientation', verbose=False)
        ori_valid_input, ori_valid_label = prepare_data(valid, nr, 'orientation', verbose=False)

        encoder = LabelEncoder()

        X_train = ori_train_input.reshape(ori_train_input.shape[0], 1, ori_train_input.shape[1])
        y_train = encoder.fit_transform(ori_train_label)
        X_valid = ori_valid_input.reshape(ori_valid_input.shape[0], 1, ori_valid_input.shape[1])
        y_valid = encoder.fit_transform(ori_valid_label)

        parameters[0]['n_classes'] = [len(encoder.classes_)]
        parameters[0]['shape'] = [X_train.shape[2]]
        parameters[0]['num_units'] = [X_train.shape[2]]

        #parameters[0]['batch_size'] = [1, 10, 100]
        #parameters[0]['batch_size'].append(X_train.shape[2])

        lstm_model = KerasClassifier(create_model, verbose=0)

        lstm = GridSearchCV(lstm_model, parameters, scoring='accuracy', cv=cross_valid,
                           verbose=verbose_level).fit(np.append(X_train, X_valid, axis=0),np.append(y_train, y_valid))

        result = (lstm.best_score_, nr, lstm.best_params_)

        if result[0] > all_ori[0]:
            all_ori = result

        K.clear_session()
    
    except ValueError as err:
        pass

In [ ]:
print(all_ori, end='\n')

### EMG

In [10]:
for nr in frame_numbers:
    print(nr)
    try:
        emg_train_input, emg_train_label = prepare_data(train, nr, 'emg', verbose=False)
        emg_valid_input, emg_valid_label = prepare_data(valid, nr, 'emg', verbose=False)

        encoder = LabelEncoder()

        X_train = emg_train_input.reshape(emg_train_input.shape[0], 1, emg_train_input.shape[1])
        y_train = encoder.fit_transform(emg_train_label)
        X_valid = emg_valid_input.reshape(emg_valid_input.shape[0], 1, emg_valid_input.shape[1])
        y_valid = encoder.fit_transform(emg_valid_label)

        parameters[0]['n_classes'] = [len(encoder.classes_)]
        parameters[0]['shape'] = [X_train.shape[2]]
        parameters[0]['num_units'] = [X_train.shape[2]]

        #parameters[0]['batch_size'] = [1, 10, 100]
        #parameters[0]['batch_size'].append(X_train.shape[2])

        lstm_model = KerasClassifier(create_model, verbose=0)

        lstm = GridSearchCV(lstm_model, parameters, scoring='accuracy', cv=cross_valid,
                           verbose=verbose_level).fit(np.append(X_train, X_valid, axis=0),np.append(y_train, y_valid))

        result = (lstm.best_score_, nr, lstm.best_params_)

        if result[0] > all_emg[0]:
            all_emg = result

        K.clear_session()
    
    except ValueError as err:
        pass

1
Fitting 1 folds for each of 9 candidates, totalling 9 fits
[CV] batch_size=1, epochs=10, n_classes=24, num_units=32, optimizer=Adam, shape=32 
[CV]  batch_size=1, epochs=10, n_classes=24, num_units=32, optimizer=Adam, shape=32, score=0.13942307692307693, total=  26.5s
[CV] batch_size=1, epochs=100, n_classes=24, num_units=32, optimizer=Adam, shape=32 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   27.0s remaining:    0.0s


[CV]  batch_size=1, epochs=100, n_classes=24, num_units=32, optimizer=Adam, shape=32, score=0.19230769230769232, total= 3.6min
[CV] batch_size=1, epochs=300, n_classes=24, num_units=32, optimizer=Adam, shape=32 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  4.1min remaining:    0.0s


[CV]  batch_size=1, epochs=300, n_classes=24, num_units=32, optimizer=Adam, shape=32, score=0.2692307692307692, total=10.7min
[CV] batch_size=10, epochs=10, n_classes=24, num_units=32, optimizer=Adam, shape=32 
[CV]  batch_size=10, epochs=10, n_classes=24, num_units=32, optimizer=Adam, shape=32, score=0.09615384615384616, total=   7.1s
[CV] batch_size=10, epochs=100, n_classes=24, num_units=32, optimizer=Adam, shape=32 
[CV]  batch_size=10, epochs=100, n_classes=24, num_units=32, optimizer=Adam, shape=32, score=0.16346153846153846, total=  26.1s
[CV] batch_size=10, epochs=300, n_classes=24, num_units=32, optimizer=Adam, shape=32 
[CV]  batch_size=10, epochs=300, n_classes=24, num_units=32, optimizer=Adam, shape=32, score=0.2403846153846154, total= 1.2min
[CV] batch_size=100, epochs=10, n_classes=24, num_units=32, optimizer=Adam, shape=32 
[CV]  batch_size=100, epochs=10, n_classes=24, num_units=32, optimizer=Adam, shape=32, score=0.038461538461538464, total=   6.7s
[CV] batch_size=100,

...

10
Fitting 1 folds for each of 9 candidates, totalling 9 fits
[CV] batch_size=1, epochs=10, n_classes=24, num_units=320, optimizer=Adam, shape=320 
[CV]  batch_size=1, epochs=10, n_classes=24, num_units=320, optimizer=Adam, shape=320, score=0.30917874396135264, total= 3.2min
[CV] batch_size=1, epochs=100, n_classes=24, num_units=320, optimizer=Adam, shape=320 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  3.2min remaining:    0.0s


[CV]  batch_size=1, epochs=100, n_classes=24, num_units=320, optimizer=Adam, shape=320, score=0.5314009661835749, total=32.1min
[CV] batch_size=1, epochs=300, n_classes=24, num_units=320, optimizer=Adam, shape=320 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed: 35.3min remaining:    0.0s


KeyboardInterrupt: 

In [ ]:
print(all_acc, end='\n')
print(all_gyro, end='\n')
print(all_ori, end='\n')
print(all_emg, end='\n')

In [ ]:
print('Duration: {}'.format(time.time() - start_time))